Create gpx constraints from a dictionary

In [1]:
def build_constraint_set(variables,constraints_as_list):
    '''build constraints
    
    Args
    ----
    variables : dict
        dictionary of variables
        key is the name
        value is the gpx variable representation
        probably a merge between the module variables and the design variables
        
    constraints_as_list : list
        the list of constraints in dict format
    
    Returns
    -------
    gpkit.ConstraintSet
    '''
    constraints = []
    for constr in constraints_as_list:
        constraints.extend(build_single_constraint(variables, constr))
    
    return constraints   

In [2]:
from gpkit import units as gp_units
def build_single_constraint(variables, constraint_as_dict):
    '''build a single constraint
    
    Args
    ----
    variables : dict
        dictionary of variables which may appear in the constraints
        key is the name as used in the constraints
        value is the gpx variable representation
        
    constraint_as_dict : dict
        the dictionary version of a constraint
    '''
    
    # check to see if there is math
    if constraint_as_dict['math'] == True:
        rhs = build_constraint_rhs(variables, constraint_as_dict['values'])
        var = variables[constraint_as_dict['name']]
        opr = constraint_as_dict['sign']
        if opr == '>=':
            return [var >= rhs]
        elif opr == '<=':
            unit = gp_units(constraint_as_dict['unit'])
            return [var <= float(rhs)*unit]
        else:
            return []
    else:
        return []
    
    

In [3]:
# moved to gpx.constraint_builder

In [4]:
import numpy as np

def build_constraint_rhs(variables, constraint_rhs):
    '''build the constraint right-hand side (RHS)
    
    Args
    ----
    variables : dict or gpkit.Model
        variables which may appear in the RHS referenced by their key
        
    constraint_rhs : list
        the list representation of a constraint
        
    Returns
    -------
    gpkit.nomials.math.Posynomial
        a gpkit representation of the constraints
    '''
    error_preamble = 'Error parsing constraint | '
    # constraint = ''
    # what is the type for a constraint?
    
    # split on "+"
    # gets to monomials
    monomials = split_list_on_char(constraint_rhs, '+')
    
    # accumulators by operations
    accl_add = []
    accl_mult = []    
    
    # for each monomial
    # split on "*" 
    
    for monterm in monomials:
        multiplication = split_list_on_char(monterm, '*')        
        
        # split each multiplication on power
        for multterm in multiplication:
            power = split_list_on_char(multterm, "^")         
            
            try:
                base = float(power[0][0])
            except:
                base = variables[str(power[0][0])]
            
                            
            if len(power) == 1:
                accl_mult.append(base)
            elif len(power) == 2:
                exponent = float(power[1][0])  # this should throw an error if it cannot be cast
                accl_mult.append(base**exponent)
            else:
                raise Exception(error_preamble + 'Power must be two terms only')
            
        accl_add.append(np.prod(accl_mult))        
        accl_mult = []
#         print(accl_add)
    return np.sum(accl_add)
            

def split_list_on_char(input_list, split_char):
    '''split an input list into a list of lists based on a char
    
    Args
    ---
    input_list : list
        single list of with 
    split_char : char
        the character where to split the list
    
    Returns
    ------
    list of lists
    '''
    # split into lists by particular value
    # following this trick: https://www.geeksforgeeks.org/python-split-list-into-lists-by-particular-value/
    size = len(input_list)    
    idx_list = [idx for idx,val in enumerate(input_list) if val == split_char]
    # if the operation is not found, just return the whole constraint
    if len(idx_list) == 0 : return [input_list]
    else:
        res = [input_list[i+1:j] for i,j in 
               zip([-1] + idx_list, 
                   idx_list + ([size] if idx_list[-1] != size else [])
                  )
              ]
        return res


## Testing

### Constraint parsing

In [5]:
sample_constraint_as_json = '''{
        "name": "Bag & Prep Std. Dev.",
        "sign": "=",
        "unit": "sec",
        "math": true,
        "value": [
          "0.25",
          "*",
          "Bag & Prep Time"
        ]
      }'''

In [6]:
sample_constraint_value_json = '["0.25","*","1.25", "+", "Bag & Prep Time", "*", "1.5", "+", "Inspection Time", "^", "2", "+", "Inspection Time"]'

In [7]:
import json

In [8]:
json.loads(sample_constraint_as_json)

{'math': True,
 'name': 'Bag & Prep Std. Dev.',
 'sign': '=',
 'unit': 'sec',
 'value': ['0.25', '*', 'Bag & Prep Time']}

In [9]:
thing = json.loads(sample_constraint_as_json)

#### test split list

In [10]:
test = json.loads(sample_constraint_value_json)
idx_list = split_list_on_char(test, "+")
print(idx_list)

# [(i+1, j) for i,j in zip([-1]+idx_list, idx_list+[len(thing)])]

[['0.25', '*', '1.25'], ['Bag & Prep Time', '*', '1.5'], ['Inspection Time', '^', '2'], ['Inspection Time']]


In [11]:
from gpkit import Variable

In [12]:
test_variables = {'Bag & Prep Time': Variable('t_{Bag & Prep}'), 
                      'Inspection Time': Variable('t_{Inspection}')
                     }

In [13]:
# test_variables = {'Bag & Prep Time': Variable('t_{Bag & Prep}'), 
#                       'Inspection Time':15
#                      }

In [14]:
test3 = build_constraint_rhs(test_variables, test)

## Understanding the types of constraints

In [15]:
alpha = Variable('\\alpha')
beta = Variable('\\beta')

In [16]:
test = [alpha >= beta]

In [17]:
type(test[0])

gpkit.nomials.math.PosynomialInequality

In [18]:
test2 = alpha + beta

In [19]:
type(test2)

gpkit.nomials.math.Posynomial